In [230]:
import requests as r
import pandas as pd
import re
import pytz
from datetime import datetime
import plotly.graph_objects as go
import time

In [131]:
url = 'https://gql.twitch.tv/gql'
headers="""Accept: */*
Accept-Encoding: gzip, deflate, br
Accept-Language: en-US
Authorization: OAuth m40f8ps9azcfwbqftpbbbkfinlvc4z
Client-Id: kimne78kx3ncx6brgo4mv6wki5h1ko
Client-Session-Id: 44767b0614275039
Client-Version: 19cbb0be-64ad-46bd-a718-ec9fb15d3e0a
Connection: keep-alive
Content-Length: 7662
Content-Type: text/plain;charset=UTF-8
Host: gql.twitch.tv
Origin: https://www.twitch.tv
Referer: https://www.twitch.tv/
Sec-Fetch-Dest: empty
Sec-Fetch-Mode: cors
Sec-Fetch-Site: same-site
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.5195.127 Safari/537.36
X-Device-Id: 9fe67639f4e1bc52"""

In [132]:
headers=dict(i.split(': ') for i in headers.split('\n'))

In [133]:
data = """[{"operationName":"UseViewCount","variables":{"channelLogin":"the4thshrine"}, "extensions":{"persistedQuery":{"version":1,"sha256Hash":"00b11c9c428f79ae228f30080a06ffd8226a1f068d6f52fbc057cbde66e994c2"}}}]"""

In [211]:
r.post(url, headers=headers, data=data).json()

[{'data': {'user': {'id': '407084579',
    'stream': {'id': '47316530253', 'viewersCount': 9, '__typename': 'Stream'},
    '__typename': 'User'}},
  'extensions': {'durationMilliseconds': 44,
   'operationName': 'UseViewCount',
   'requestID': '01GFJ6YJ6JDRHD10N47SKR7AHJ'}}]

In [207]:
def get_payload(name_of_the_channel, get_counts, get_general_info = False, get_preview_image = False, get_stream_info = False):
  x = {"operationName":"UseViewCount","variables":{"channelLogin":name_of_the_channel}, "extensions":{"persistedQuery":{"version":1,"sha256Hash":"00b11c9c428f79ae228f30080a06ffd8226a1f068d6f52fbc057cbde66e994c2"}}}
  y = {"operationName":"PlayerTrackingContextQuery","variables":{"channel":name_of_the_channel,"isLive":True,"hasCollection":False,"collectionID":"","videoID":"","hasVideo":False,"slug":"","hasClip":False},"extensions":{"persistedQuery":{"version":1,"sha256Hash":"3fbf508886ff5e008cb94047acc752aad7428c07b6055995604de16c4b01160a"}}}
  z = {"operationName":"VideoPreviewOverlay","variables":{"login":name_of_the_channel},"extensions":{"persistedQuery":{"version":1,"sha256Hash":"3006e77e51b128d838fa4e835723ca4dc9a05c5efd4466c1085215c6e437e65c"}}}
  t = {"operationName":"StreamMetadata","variables":{"channelLogin":name_of_the_channel},"extensions":{"persistedQuery":{"version":1,"sha256Hash":"a647c2a13599e5991e175155f798ca7f1ecddde73f7f341f39009c14dbf59962"}}}
  pre = """[{},{},{},{}]""".format(str(x)*get_counts, str(y)*get_general_info, str(z)*get_preview_image, str(t)*get_stream_info).replace("'",'"')
  pre = re.sub(',+',',',pre)
  pre = re.sub(',+]$',']',pre)
  return re.sub('^\[','[',pre)

In [218]:
get_payload('the4thshrine', get_counts=True)

'[{"operationName": "UseViewCount", "variables": {"channelLogin": "the4thshrine"}, "extensions": {"persistedQuery": {"version": 1, "sha256Hash": "00b11c9c428f79ae228f30080a06ffd8226a1f068d6f52fbc057cbde66e994c2"}}}]'

In [219]:
def clean_a_bit(data: r.models.Response) -> pd.Series(dtype = 'object'):
  data = data.json()
  namedData = dict()
  cleanedData = dict()
  for i in data:
    name = i['extensions']['operationName']
    namedData[name] = i['data']
  now = datetime.now(tz =pytz.timezone('America/New_York'))

  if 'PlayerTrackingContextQuery' in namedData.keys():
    cleanedData['streamer_id'] = namedData['PlayerTrackingContextQuery']['user']['id']
    cleanedData['streamer_login'] = namedData['PlayerTrackingContextQuery']['user']['login']
    cleanedData['game_id'] = namedData['PlayerTrackingContextQuery']['user']['broadcastSetting']['id']
    cleanedData['name'] = namedData['PlayerTrackingContextQuery']['user']['broadcastSetting']['title']
    cleanedData['type'] = namedData['PlayerTrackingContextQuery']['user']['broadcastSetting']['__typename']
  if 'VideoPreviewOverlay' in namedData.keys():
    cleanedData['PreviewImage'] = namedData['VideoPreviewOverlay']['user']['stream']['PreviewImage']
  if 'StreamMetaData' in namedData.keys():
    cleanedData['lastBroadcast_title'] = namedData['StreamMetaData']['user']['lastBroadcast']['title']
    cleanedData['current_created_at'] = namedData['StreamMetaData']['user']['stream']['createdAt']
  if 'UseViewCount' in namedData.keys():
    cleanedData['current_viewers'] = namedData['UseViewCount']['user']['stream']['viewersCount']
  series = pd.Series(cleanedData)
  series['time']=now
  return series.to_frame().T.set_index('time')

In [220]:
def get_data(username, get_counts, get_general_info=False, get_preview_image=False, get_stream_info=False, **kwargs):
  url = 'https://gql.twitch.tv/gql'
  headers = """Accept: */*
Accept-Encoding: gzip, deflate, br
Accept-Language: en-US
Authorization: OAuth m40f8ps9azcfwbqftpbbbkfinlvc4z
Client-Id: kimne78kx3ncx6brgo4mv6wki5h1ko
Client-Session-Id: 44767b0614275039
Client-Version: 19cbb0be-64ad-46bd-a718-ec9fb15d3e0a
Connection: keep-alive
Content-Length: 7662
Content-Type: text/plain;charset=UTF-8
Host: gql.twitch.tv
Origin: https://www.twitch.tv
Referer: https://www.twitch.tv/
Sec-Fetch-Dest: empty
Sec-Fetch-Mode: cors
Sec-Fetch-Site: same-site
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.5195.127 Safari/537.36
X-Device-Id: 9fe67639f4e1bc52""".format(username)
  headers = dict(i.split(': ') for i in headers.split('\n'))
  resp=r.post(url, headers=headers, data=get_payload(username,get_counts,get_preview_image,get_stream_info))
  return clean_a_bit(resp)

In [228]:
get_data('the4thshrine', get_counts=True)

,current_viewers
time,
2022-10-17 01:55:32.836606-04:00,5


In [231]:
data = get_data('the4thshrine', get_counts=True)
def create_plot(data):
  fig = go.FigureWidget([go.Scatter(x = data.index, y = data.current_viewers,
                                    mode = 'lines')])
  return fig
fig = create_plot(data)
fig.show()

In [226]:
def update(data, fig):
  dat = create_plot(data).data
  fig.data[0].x = dat[0].x
  fig.data[0].y = dat[0].y

In [232]:
while True:
  df = get_data('the4thshrine', get_counts=True)
  data = data.append(df)
  time.sleep(20)
  update(data,fig)
  print('Done!')

Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!


KeyboardInterrupt: ignored

In [233]:
fig.show()